In [8]:
import os
from collections import Counter
from PIL import Image

BASE = "/Users/dhruvsharma/Downloads/DS - Brain Tumor MRI Image Classification/Tumour"
splits = ["train", "valid", "test"]
for split in splits:
    path = os.path.join(BASE, split)
    counts = Counter(d for d in os.listdir(path) if os.path.isdir(os.path.join(path,d)))
    print(f"{split.upper():6}", counts)

    # show one sample
    cls = next(iter(counts))
    img = Image.open(os.path.join(path, cls, os.listdir(os.path.join(path,cls))[0]))
    print(f"  Sample {split}/{cls} → size={img.size}, mode={img.mode}")


TRAIN  Counter({'pituitary': 1, 'No Tumor': 1, 'glioma': 1, 'meningioma': 1})
  Sample train/pituitary → size=(640, 640), mode=RGB
VALID  Counter({'pituitary': 1, 'No Tumor': 1, 'glioma': 1, 'meningioma': 1})
  Sample valid/pituitary → size=(640, 640), mode=RGB
TEST   Counter({'pituitary': 1, 'No Tumor': 1, 'glioma': 1, 'meningioma': 1})
  Sample test/pituitary → size=(640, 640), mode=RGB


In [9]:
# Data Preprocessing functions
import tensorflow as tf

IMG_SIZE = (224,224)

def preprocess_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = img / 255.0
    return img

def make_dataset(split_dir, batch=32, augment=False):
    ds = tf.keras.preprocessing.image_dataset_from_directory(
        split_dir,
        image_size=IMG_SIZE,
        batch_size=batch,
        label_mode='categorical'
    )
    return ds


In [10]:
# Data Augmentation

from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
    layers.RandomTranslation(0.1,0.1),
])


In [11]:
from tensorflow.keras import layers, models

def build_custom_cnn(input_shape=(224,224,3), num_classes=4):
    model = models.Sequential([
        layers.Input(input_shape),
        layers.Conv2D(32,3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        layers.Conv2D(64,3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        layers.Conv2D(128,3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPool2D(),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


In [12]:
import os
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from preprocessing import make_dataset
#from model_custom import build_custom_cnn

train_ds = make_dataset(os.path.join(BASE,"train"), batch=32, augment=True)
val_ds   = make_dataset(os.path.join(BASE,"valid"), batch=32)

model = build_custom_cnn()
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("models/custom_cnn.keras", save_best_only=True)
]
history = model.fit(train_ds, validation_data=val_ds, epochs=50, callbacks=callbacks)


Found 1695 files belonging to 4 classes.
Found 502 files belonging to 4 classes.
Epoch 1/50


2025-07-24 20:44:04.593388: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


53/53 ━━━━━━━━━━━━━━━━━━━━ 13s 227ms/step - accuracy: 0.6695 - loss: 1.0819 - val_accuracy: 0.1972 - val_loss: 10.1269
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 225ms/step - accuracy: 0.8428 - loss: 0.4722 - val_accuracy: 0.4841 - val_loss: 1.2969
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 220ms/step - accuracy: 0.9033 - loss: 0.2660 - val_accuracy: 0.5996 - val_loss: 1.0097
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 219ms/step - accuracy: 0.9179 - loss: 0.2126 - val_accuracy: 0.8367 - val_loss: 0.4459
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 228ms/step - accuracy: 0.9546 - loss: 0.1361 - val_accuracy: 0.8526 - val_loss: 0.4173
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 223ms/step - accuracy: 0.9580 - loss: 0.1155 - val_accuracy: 0.8147 - val_loss: 0.4962
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 221ms/step - accuracy: 0.9636 - loss: 0.1019 - val_accuracy: 0.8805 - val_loss: 0.3454
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 12s 221ms/step - accuracy: 0.9873 - loss: 0.0495 - val_accuracy: 0.93

In [13]:
## Model Transfer
import tensorflow as tf
from tensorflow.keras import layers, models

def build_transfer_model(base_name='ResNet50', input_shape=(224,224,3), num_classes=4):
    base = getattr(tf.keras.applications, base_name)(
        include_top=False, weights='imagenet', input_shape=input_shape, pooling='avg'
    )
    base.trainable = False

    x = layers.Input(shape=input_shape)
    y = base(x, training=False)
    y = layers.Dense(128, activation='relu')(y)
    y = layers.BatchNormalization()(y)
    y = layers.Dropout(0.5)(y)
    output = layers.Dense(num_classes, activation='softmax')(y)

    model = models.Model(x, output)
    model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"])

    return model


In [14]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from preprocessing import make_dataset
#from model_transfer import build_transfer_model

train_ds = make_dataset(os.path.join(BASE,"train"), batch=32, augment=True)
val_ds   = make_dataset(os.path.join(BASE,"valid"), batch=32)

model = build_transfer_model('EfficientNetB0')
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint("models/transfer_efficientnet.keras", save_best_only=True)
]
model.fit(train_ds, validation_data=val_ds, epochs=40, callbacks=callbacks)


Found 1695 files belonging to 4 classes.
Found 502 files belonging to 4 classes.
Epoch 1/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 16s 237ms/step - accuracy: 0.6302 - loss: 1.1127 - val_accuracy: 0.8526 - val_loss: 0.3899
Epoch 2/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.8063 - loss: 0.5162 - val_accuracy: 0.8247 - val_loss: 0.4436
Epoch 3/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.8621 - loss: 0.3595 - val_accuracy: 0.8765 - val_loss: 0.3639
Epoch 4/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 189ms/step - accuracy: 0.8846 - loss: 0.3308 - val_accuracy: 0.8944 - val_loss: 0.2867
Epoch 5/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 185ms/step - accuracy: 0.8914 - loss: 0.2945 - val_accuracy: 0.9024 - val_loss: 0.2898
Epoch 6/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.9118 - loss: 0.2672 - val_accuracy: 0.9084 - val_loss: 0.2609
Epoch 7/40
53/53 ━━━━━━━━━━━━━━━━━━━━ 10s 186ms/step - accuracy: 0.9046 - loss: 0.2624 - val_accuracy: 0.9104 - val_loss: 0.2319
Epoch 8/40
53/53

In [15]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 1280)           │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,544,049 (17.33 MB)

 Trainable params: 164,740 (643.52 KB)

 Non-trainable params: 4,049,827 (15.45 MB)

 Optimizer params: 329,482 (1.26 MB)

In [16]:
#Training and monitoring
import matplotlib.pyplot as plt

def plot_history(hist, name):
    plt.plot(hist.history['loss'], label='train_loss')
    plt.plot(hist.history['val_loss'], label='val_loss')
    plt.title(name)
    plt.legend()
    plt.show()


In [17]:
#Evaluation
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
#from preprocessing import make_dataset

test_ds = make_dataset(os.path.join(BASE,"test"), batch=32, augment=False)
model = load_model("models/transfer_efficientnet.keras")
y_true = np.concatenate([y for _,y in test_ds], axis=0)
y_pred = model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_true, axis=1)

print(classification_report(y_true_labels, y_pred_labels, target_names=test_ds.class_names))
print(confusion_matrix(y_true_labels, y_pred_labels))


Found 246 files belonging to 4 classes.


2025-07-24 20:50:42.415947: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


8/8 ━━━━━━━━━━━━━━━━━━━━ 4s 323ms/step
              precision    recall  f1-score   support

    No Tumor       0.15      0.14      0.14        49
      glioma       0.35      0.39      0.37        80
  meningioma       0.24      0.21      0.22        63
   pituitary       0.24      0.24      0.24        54

    accuracy                           0.26       246
   macro avg       0.24      0.24      0.24       246
weighted avg       0.26      0.26      0.26       246

[[ 7 18 13 11]
 [10 31 16 23]
 [20 22 13  8]
 [11 17 13 13]]


In [18]:
#Evaluation by Custom CNN
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import load_model
#from preprocessing import make_dataset

test_ds = make_dataset(os.path.join(BASE,"test"), batch=32, augment=False)
model = load_model("models/custom_cnn.keras")
y_true = np.concatenate([y for _,y in test_ds], axis=0)
y_pred = model.predict(test_ds)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_true, axis=1)

print(classification_report(y_true_labels, y_pred_labels, target_names=test_ds.class_names))
print(confusion_matrix(y_true_labels, y_pred_labels))


Found 246 files belonging to 4 classes.
1/8 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step

2025-07-24 20:50:46.663090: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
              precision    recall  f1-score   support

    No Tumor       0.15      0.14      0.14        49
      glioma       0.37      0.38      0.37        80
  meningioma       0.27      0.27      0.27        63
   pituitary       0.33      0.33      0.33        54

    accuracy                           0.29       246
   macro avg       0.28      0.28      0.28       246
weighted avg       0.29      0.29      0.29       246

[[ 7 19 12 11]
 [15 30 19 16]
 [17 19 17 10]
 [ 9 13 14 18]]


In [20]:
import streamlit as st
import numpy as np
from PIL import Image
from tensorflow.keras.models import load_model

CLASS_NAMES = ["glioma","meningioma","no_tumor","pituitary"]
model = load_model("models/transfer_efficientnet.keras")

st.title("🧠 Brain Tumor MRI Classifier")
uploaded = st.file_uploader("Upload an MRI image", type=["png","jpg","jpeg"])
if uploaded:
    img = Image.open(uploaded).convert("RGB").resize((224,224))
    x = np.array(img)/255.0
    preds = model.predict(x[np.newaxis,...])[0]
    idx = np.argmax(preds)
    st.write(f"**Prediction:** {CLASS_NAMES[idx]} ({preds[idx]*100:.1f}% confidence)")
    st.bar_chart(preds, use_container_width=True)



2025-07-24 20:53:04.571 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.614 
  command:

    streamlit run /Users/dhruvsharma/PyCharmMiscProject/.venv/lib/python3.9/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-07-24 20:53:04.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.614 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:04.615 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-24 20:53:0

In [22]:
streamlit run /Users/dhruvsharma/PyCharmMiscProject/.venv/lib/python3.9/site-packages/ipykernel_launcher.py

SyntaxError: invalid syntax (2903271827.py, line 1)